In [1]:
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.manifold import TSNE
import seaborn as sns
from tqdm import tqdm
import time
from numba import jit, autojit, prange

import tools
from kernel import linear_kernel
from algorithm import *
from tools import *

folder_name = "data"

In [2]:
file_idx = 1

train_file_name = "Xtr" + str(file_idx) + ".csv"
result_file_name = "Ytr"+ str(file_idx) +".csv"
test_file_name = "Xte" + str(file_idx) + ".csv"

In [3]:
"""
training_percentage = 0.9
data_array = tools.read_file(os.path.join(folder_name, train_file_name))
result_array = tools.read_file(os.path.join(folder_name, result_file_name))
training_nb = round(training_percentage*len(data_array))
training_array = data_array[:training_nb]
training_labels = result_array[:training_nb]
test_array = data_array[training_nb:]
test_labels = result_array[training_nb:]
"""
data_array_train = tools.read_file(os.path.join(folder_name, train_file_name))
data_array_test = tools.read_file(os.path.join(folder_name, test_file_name))

In [4]:
print(data_array_train.shape, data_array_test.shape)
data_array = np.concatenate((data_array_train, data_array_test))
print(data_array.shape)

(2000, 100) (1000, 100)
(3000, 100)


In [5]:
def compute_kernel_cell(str_a, str_b, k, lmb):
    len_a = len(str_a)
    len_b = len(str_b)
    
    # Computing the recursive side function B
    rec_val = np.zeros((k+1, len_a+1, len_b+1))
    rec_val[0, :, :] = np.ones((len_a+1, len_b+1))
    # Initialization
    #for cur_k in range(1, k+1):
    #    for cur_i in range(len_a+1):
    #        for cur_j in range(len_b+1):
    #            if (min(cur_i, cur_j) < cur_k):
    #                rec_val[cur_k, cur_i, cur_j] = 0
    # Completing the array of the recursion function
    for cur_k in range(1, k+1):
        for cur_i in range(cur_k, len_a+1):
            for cur_j in range(cur_k, len_b+1):
                rec_val[cur_k, cur_i, cur_j] = lmb*(rec_val[cur_k, cur_i-1, cur_j] + rec_val[cur_k, cur_i, cur_j-1] - lmb*rec_val[cur_k, cur_i-1, cur_j-1])
                if (str_a[cur_i-1] == str_b[cur_j-1]):
                    rec_val[cur_k, cur_i, cur_j] += lmb*lmb*rec_val[cur_k-1, cur_i-1, cur_j-1]
    
    # Computing the final value
    ker_val = np.zeros((len_a+1, len_b+1))
    # Initialization
    #for cur_i in range(len_a+1):
    #    for cur_j in range(len_b+1):
    #        if (min(cur_i, cur_j) < k):
    #            ker_val[cur_i, cur_j] = 0
    # Finishing
    for cur_i in range(k-1, len_a):
        sec_term = 0
        for cur_j in range(1, k+1):
            if (str_b[cur_j-1] == str_a[cur_i]):
                sec_term += rec_val[k-1, cur_i, cur_j-1]
        ker_val[cur_i+1, k] = ker_val[cur_i, k] + lmb*lmb*sec_term
    for cur_j in range(k, len_b):
        sec_term = 0
        for cur_i in range(1, len_a+1):
            if (str_a[cur_i-1] == str_b[cur_j]):
                sec_term += rec_val[k-1, cur_i-1, cur_j]
        ker_val[len_a, cur_j+1] = ker_val[len_a, cur_j] + lmb*lmb*sec_term
        
    return ker_val[len_a, len_b]

In [6]:
@jit(nopython=True, fastmath=True)
def compute_B(str_a, str_b, len_a, len_b, k, lmb, rec_val):
    for cur_k in range(1, k+1):
        for cur_i in range(cur_k, len_a+1):
            for cur_j in range(cur_k, len_b+1):
                rec_val[cur_k, cur_i, cur_j] = lmb*(rec_val[cur_k, cur_i-1, cur_j] + rec_val[cur_k, cur_i, cur_j-1] - lmb*rec_val[cur_k, cur_i-1, cur_j-1])
                if (str_a[cur_i-1] == str_b[cur_j-1]):
                    rec_val[cur_k, cur_i, cur_j] += lmb*lmb*rec_val[cur_k-1, cur_i-1, cur_j-1]
    return rec_val

@jit(nopython=True, fastmath=True)
def compute_kernel_cell2(str_a, str_b, k, lmb):
    len_a = len(str_a)
    len_b = len(str_b)
    
    # Computing the recursive side function B
    rec_val = np.zeros((k+1, len_a+1, len_b+1))
    rec_val[0, :, :] = np.ones((len_a+1, len_b+1))
    # Initialization
    #for cur_k in range(1, k+1):
    #    for cur_i in range(len_a+1):
    #        for cur_j in range(len_b+1):
    #            if (min(cur_i, cur_j) < cur_k):
    #                rec_val[cur_k, cur_i, cur_j] = 0
    # Completing the array of the recursion function
    rec_val = compute_B(str_a, str_b, len_a, len_b, k, lmb, rec_val)
    
    # Computing the final value
    ker_val = np.zeros((len_a+1, len_b+1))
    # Initialization
    #for cur_i in range(len_a+1):
    #    for cur_j in range(len_b+1):
    #        if (min(cur_i, cur_j) < k):
    #            ker_val[cur_i, cur_j] = 0
    # Finishing
    for cur_i in range(k-1, len_a):
        sec_term = 0
        for cur_j in range(1, k+1):
            if (str_b[cur_j-1] == str_a[cur_i]):
                sec_term += rec_val[k-1, cur_i, cur_j-1]
        ker_val[cur_i+1, k] = ker_val[cur_i, k] + lmb*lmb*sec_term
    for cur_j in range(k, len_b):
        sec_term = 0
        for cur_i in range(1, len_a+1):
            if (str_a[cur_i-1] == str_b[cur_j]):
                sec_term += rec_val[k-1, cur_i-1, cur_j]
        ker_val[len_a, cur_j+1] = ker_val[len_a, cur_j] + lmb*lmb*sec_term
        
    return ker_val[len_a, len_b]

In [7]:
@jit(nopython=True, parallel=True)
def compute_kernel(data_array, k , lmb):
    str_nb = data_array.shape[0]
    kernel_array = np.zeros((str_nb, str_nb))
    for i in prange(str_nb):
        for j in range(i, str_nb):
            kernel_value = compute_kernel_cell2(data_array[i], data_array[j], k, lmb)
            kernel_array[i,j] = kernel_value
            kernel_array[j,i] = kernel_value
    return kernel_array

In [8]:
def transform_array(data_array):
    res_array = np.zeros((len(data_array), len(data_array[0])))
    for str_idx in range(len(data_array)):
        for idx in range(len(data_array[0])):
            res_array[str_idx, idx] = ord(data_array[str_idx][idx])
    return res_array

In [9]:
reduced_array = data_array
print(reduced_array.shape)
str_nb = len(reduced_array)

reduced_array = transform_array(reduced_array)

(3000, 100)


In [10]:
time1 = time.time()
#compute_kernel_cell("car", "car", 2, 2)
value = compute_kernel_cell(reduced_array[0], reduced_array[1], 3, 0.8)
time2 = time.time()
# renvoie 2*lmb**4 + lmb**6
print(value)
print(time2-time1)
# entre 0.43 et 0.48 globalement

print("")

time1 = time.time()
#compute_kernel_cell("car", "car", 2, 2)
value = compute_kernel_cell2(reduced_array[0], reduced_array[1], 3, 0.8)
time2 = time.time()
print(value)
print(time2-time1)

16065.34541054997
0.08600664138793945

16065.345410549973
1.8003623485565186


In [11]:
k = 8
lmb = 0.3

In [12]:
time1 = time.time()
res_array = compute_kernel(reduced_array[:2], k, lmb)
time2 = time.time()
print(time2-time1)

2.064072608947754


In [13]:
time1 = time.time()
res_array = compute_kernel(reduced_array, k, lmb)
time2 = time.time()
print(time2-time1)

1787.5379769802094


In [20]:
print(res_array)

[[2.38623124e-06 3.75101861e-08 1.97616336e-07 ... 1.63929920e-07
  3.29597293e-08 4.82178404e-08]
 [3.75101861e-08 2.14945642e-06 8.91291070e-08 ... 5.89230820e-08
  8.88915229e-08 1.44057608e-07]
 [1.97616336e-07 8.91291070e-08 1.90626012e-06 ... 7.56236287e-08
  1.10351668e-07 5.96786747e-08]
 ...
 [1.63929920e-07 5.89230820e-08 7.56236287e-08 ... 3.25025407e-06
  1.31979038e-07 1.05519245e-07]
 [3.29597293e-08 8.88915229e-08 1.10351668e-07 ... 1.31979038e-07
  2.41864618e-06 6.26802618e-08]
 [4.82178404e-08 1.44057608e-07 5.96786747e-08 ... 1.05519245e-07
  6.26802618e-08 2.42900451e-06]]


In [14]:
save_res = res_array

In [16]:
print(res_array.shape)

(3000, 3000)


In [17]:
np.save("substring_test"+str(file_idx)+".npy", res_array)

# Other stuff, for testing purposes

In [ ]:
str_a = "car"
str_b = "car"
k=2
lmb=2
compute_kernel_cell2(str_a, str_b, k, lmb)

In [ ]:
k = 3
lmb = 0.5

#test_arr = np.array([['c','a','r'], ['b','a','t']])
test_arr = data_array[[0, 1999], :]
test_arr = transform_array(test_arr)
print(test_arr)
ker = compute_kernel(test_arr, 3, lmb)
print(ker)

In [ ]:
file_idx_test = 2
thingy = np.load("substring_test"+str(file_idx_test)+".npy")
print(thingy)
print(thingy.max(), thingy.min())

In [ ]:
file_idx = 0

train_file_name = "Xtr" + str(file_idx) + ".csv"
result_file_name = "Ytr"+ str(file_idx) +".csv"
test_file_name = "Xte" + str(file_idx) + ".csv"

training_percentage = 0.9
data_array = tools.read_file(os.path.join(folder_name, train_file_name))
#data_array = [data_str.replace('\n', '') for data_str in data_array]
result_array = tools.read_file(os.path.join(folder_name, result_file_name))
training_nb = round(training_percentage*len(data_array))
training_array = data_array[:training_nb]
training_labels = result_array[:training_nb]
test_array = data_array[training_nb:]
test_labels = result_array[training_nb:]

print(test_labels)